# Kite Connect Testing Notebook

Test suite for Kite API integration, position analysis, and trading operations.

## 1. Import Dependencies and Initialize Kite

In [5]:
# In your main_strategy.py
from pprint import pprint
import pandas as pd
import numpy as np
from datetime import datetime, date

In [ ]:
# Reload the module to get fresh kite object with new token
import importlib
import load_kite_from_access
#importlib.reload(load_kite_from_access)
kite = load_kite_from_access.kite

# Check kite status and get UserID
print(f"Kite object: {kite}")
print(f"Type: {type(kite)}")
if kite:
    try:
        profile = kite.profile()
        print(f"\n✅ UserID: {profile['user_id']}")
        print(f"✅ User Name: {profile['user_name']}")
        print(f"✅ Email: {profile['email']}")
    except Exception as e:
        print(f"Error getting profile: {e}")
else:
    print("\n❌ Kite object is None - session not established")
    print("Please ensure you have run config.py to generate access token")

✅ Connected: Indhuja . (ID: XJY521)
Kite object: <kiteconnect.connect.KiteConnect object at 0x0000022C82617250>
Type: <class 'kiteconnect.connect.KiteConnect'>

✅ UserID: XJY521
✅ User Name: Indhuja .
✅ Email: sathyakumarnandakumar@gmail.com


## 2. Get User Profile

In [11]:

# The kite object is already initialized and validated!
if kite:
    print(kite)
    print("Kite connection successful!")
    profile = kite.profile()
    user_id = profile['user_id']
    user_name = profile['user_name']
    print(f"Connected as User ID: {user_id}")
    print(f"User Name: {user_name}")

## 3. Fetch and Group Instruments (NSE + NFO)

In [17]:

try:
    # 1. Fetch instruments from NSE (Equities & Indices) and NFO (Futures & Options)
    print("Fetching instruments from NSE...")
    instruments_nse = kite.instruments("NSE")
    
    print("Fetching instruments from NFO...")
    instruments_nfo = kite.instruments("NFO")
    
    print("Fetching instruments from MCX...")
    instruments_mcx = kite.instruments("MCX")

    # 2. Combine and convert to DataFrame
    instruments_all = instruments_nse + instruments_nfo 
    
    df_all = pd.DataFrame(instruments_all)
    
    
    print(f"\nTotal Instruments Fetched: {len(df_all):,}")
    print(f"  - NSE: {len(instruments_nse):,}")
    print(f"  - NFO: {len(instruments_nfo):,}")
    print(f"\nInstrument Types: {sorted(df_all['instrument_type'].unique())}")
    print(f"Segments: {sorted(df_all['segment'].unique())}")
    
    # 3. Group instruments by type
    # Equity (EQ) - from NSE
    df_equity = df_all[(df_all['instrument_type'] == 'EQ') & (df_all['segment'] == 'NSE')].copy()
    
    # Index - INDICES segment
    df_index = df_all[df_all['segment'] == 'INDICES'].copy()
    
    # Futures (FUT) - from NFO
    df_futures = df_all[df_all['instrument_type'] == 'FUT'].copy()
    
    # Options - Call (CE)
    df_options_ce = df_all[df_all['instrument_type'] == 'CE'].copy()
    
    # Options - Put (PE)
    df_options_pe = df_all[df_all['instrument_type'] == 'PE'].copy()
    
    # 4. Display summary
    print("\n" + "="*60)
    print("INSTRUMENT GROUPING SUMMARY")
    print("="*60)
    print(f"Equity (EQ):          {len(df_equity):,} instruments")
    print(f"Indices:              {len(df_index):,} instruments")
    print(f"Futures (FUT):        {len(df_futures):,} instruments")
    print(f"Options - Call (CE):  {len(df_options_ce):,} instruments")
    print(f"Options - Put (PE):   {len(df_options_pe):,} instruments")
    print("="*60)
    
    # 5. Show samples of each type
    print("\n EQUITY SAMPLE (Top 5):")
    if len(df_equity) > 0:
        print(df_equity[['tradingsymbol', 'name', 'exchange', 'instrument_type']].head())
    
    print("\n INDEX SAMPLE (Top 5):")
    if len(df_index) > 0:
        print(df_index[['tradingsymbol', 'name', 'exchange']].head())
    
    print("\n FUTURES SAMPLE (Top 5):")
    if len(df_futures) > 0:
        print(df_futures[['tradingsymbol', 'name', 'expiry', 'lot_size', 'instrument_type']].head())
    
    print("\n OPTIONS CE SAMPLE (Top 5):")
    if len(df_options_ce) > 0:
        print(df_options_ce[['tradingsymbol', 'name', 'strike', 'expiry', 'lot_size']].head())
    
    print("\n OPTIONS PE SAMPLE (Top 5):")
    if len(df_options_pe) > 0:
        print(df_options_pe[['tradingsymbol', 'name', 'strike', 'expiry', 'lot_size']].head())
    
    print("\n✅ All instruments grouped and stored in pandas DataFrames:")
    print("   - df_equity, df_index, df_futures, df_options_ce, df_options_pe")

except Exception as e:
    print(f"Error occurred: {e}")
    import traceback
    traceback.print_exc()

Fetching instruments from NSE...
Fetching instruments from NFO...
Fetching instruments from MCX...

Total Instruments Fetched: 48,685
  - NSE: 9,192
  - NFO: 39,493

Instrument Types: ['CE', 'EQ', 'FUT', 'PE']
Segments: ['INDICES', 'NFO-FUT', 'NFO-OPT', 'NSE']

INSTRUMENT GROUPING SUMMARY
Equity (EQ):          9,058 instruments
Indices:              134 instruments
Futures (FUT):        633 instruments
Options - Call (CE):  19,469 instruments
Options - Put (PE):   19,391 instruments

 EQUITY SAMPLE (Top 5):
    tradingsymbol                       name exchange instrument_type
134   GOLDSTAR-SM             GOLDSTAR POWER      NSE              EQ
135    21STCENMGM  21ST CENTURY MGMT SERVICE      NSE              EQ
136      AARTIIND           AARTI INDUSTRIES      NSE              EQ
137           ABB                  ABB INDIA      NSE              EQ
138    656KA30-SG          SDL KA 6.56% 2030      NSE              EQ

 INDEX SAMPLE (Top 5):
       tradingsymbol               name exc

In [16]:
# Convert to DataFrame and show unique values
df_mcx = pd.DataFrame(instruments_mcx)

print("="*80)
print("MCX INSTRUMENTS - UNIQUE VALUES")
print("="*80)

print(f"\nTotal MCX Instruments: {len(df_mcx)}")

print(f"\n📊 Unique Instrument Types:")
print(df_mcx['instrument_type'].unique())

print(f"\n📊 Unique Segments:")
print(df_mcx['segment'].unique())

print(f"\n📊 Unique Exchanges:")
print(df_mcx['exchange'].unique())

print(f"\n📊 Unique Names (Commodities):")
print(sorted(df_mcx['name'].unique()))

print(f"\n📊 Total Unique Commodities: {df_mcx['name'].nunique()}")

# Display sample data
print(f"\n{'='*80}")
print("SAMPLE MCX INSTRUMENTS:")
print("="*80)
print(df_mcx[['tradingsymbol', 'name', 'instrument_type', 'expiry', 'lot_size']].head(10))

MCX INSTRUMENTS - UNIQUE VALUES

Total MCX Instruments: 73774

📊 Unique Instrument Types:
['EQ' 'FUT' 'CE' 'PE']

📊 Unique Segments:
['INDICES' 'MCX-FUT' 'MCX-OPT']

📊 Unique Exchanges:
['MCX']

📊 Unique Names (Commodities):
['ALUMINI', 'ALUMINIUM', 'CARDAMOM', 'COPPER', 'COTTON', 'COTTONOIL', 'CRUDEOIL', 'CRUDEOILM', 'ELECDMBL', 'GOLD', 'GOLDGUINEA', 'GOLDM', 'GOLDPETAL', 'GOLDTEN', 'KAPAS', 'LEAD', 'LEADMINI', 'MCX INDEX MCXCOMDEX', 'MCXAGRI', 'MCXBULLDEX', 'MCXCOMPDEX', 'MCXCOPRDEX', 'MCXCRUDEX', 'MCXENERGY', 'MCXGOLDEX', 'MCXMETAL', 'MCXMETLDEX', 'MCXSILVDEX', 'MENTHAOIL', 'NATGASMINI', 'NATURALGAS', 'NICKEL', 'SILVER', 'SILVERM', 'SILVERMIC', 'STEELREBAR', 'ZINC', 'ZINCMINI']

📊 Total Unique Commodities: 38

SAMPLE MCX INSTRUMENTS:
  tradingsymbol        name instrument_type expiry  lot_size
0     MCXGOLDEX   MCXGOLDEX              EQ                1
1    MCXMETLDEX  MCXMETLDEX              EQ                1
2     MCXCRUDEX   MCXCRUDEX              EQ                1
3    MCXC

## 4. Export Instruments to CSV Files

In [22]:
# Standardize and format all dataframes for consistency
print("="*80)
print("STANDARDIZING DATA FORMATS")
print("="*80)

# Helper function to standardize datetime columns
def standardize_dataframe(df, name="DataFrame"):
    df_clean = df.copy()
    
    # Convert date columns to datetime64
    date_columns = ['expiry', 'last_date']
    for col in date_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
    
    # Ensure numeric columns are proper types
    numeric_columns = ['strike', 'tick_size', 'lot_size']
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # String columns should be proper strings (including tokens)
    string_columns = ['tradingsymbol', 'name', 'exchange', 'segment', 'instrument_type', 
                      'instrument_token', 'exchange_token']
    for col in string_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str)
    
    print(f"✅ {name}: {len(df_clean):,} rows standardized")
    return df_clean

# Standardize all NSE/NFO dataframes
print("\n📊 NSE/NFO DataFrames:")
df_all = standardize_dataframe(df_all, "df_all")
df_equity = standardize_dataframe(df_equity, "df_equity")
df_index = standardize_dataframe(df_index, "df_index")
df_futures = standardize_dataframe(df_futures, "df_futures")
df_options_ce = standardize_dataframe(df_options_ce, "df_options_ce")
df_options_pe = standardize_dataframe(df_options_pe, "df_options_pe")

# Standardize MCX dataframe
print("\n📊 MCX DataFrames:")
df_mcx = standardize_dataframe(df_mcx, "df_mcx")

# Create and standardize MCX filtered dataframes
df_mcx_futures = df_mcx[df_mcx['instrument_type'] == 'FUT'].copy()
df_mcx_ce = df_mcx[df_mcx['instrument_type'] == 'CE'].copy()
df_mcx_pe = df_mcx[df_mcx['instrument_type'] == 'PE'].copy()

print(f"✅ df_mcx_futures: {len(df_mcx_futures):,} rows")
print(f"✅ df_mcx_ce: {len(df_mcx_ce):,} rows")
print(f"✅ df_mcx_pe: {len(df_mcx_pe):,} rows")

print("\n" + "="*80)
print("DATA STANDARDIZATION COMPLETE")
print("="*80)
print("All dataframes now have:")
print("  - DateTime columns in datetime64 format")
print("  - Numeric columns (strike, tick_size, lot_size) in proper numeric types")
print("  - String columns (including instrument_token, exchange_token) as strings")
print("  - Ready for Parquet export!")

STANDARDIZING DATA FORMATS

📊 NSE/NFO DataFrames:
✅ df_all: 48,685 rows standardized
✅ df_equity: 9,058 rows standardized
✅ df_index: 134 rows standardized
✅ df_futures: 633 rows standardized
✅ df_options_ce: 19,469 rows standardized
✅ df_options_pe: 19,391 rows standardized

📊 MCX DataFrames:
✅ df_mcx: 73,774 rows standardized
✅ df_mcx_futures: 145 rows
✅ df_mcx_ce: 36,809 rows
✅ df_mcx_pe: 36,809 rows

DATA STANDARDIZATION COMPLETE
All dataframes now have:
  - DateTime columns in datetime64 format
  - Numeric columns (strike, tick_size, lot_size) in proper numeric types
  - String columns (including instrument_token, exchange_token) as strings
  - Ready for Parquet export!


In [23]:
# Save all DataFrames to Parquet files for efficient storage
import os

# Create a folder for the data
output_folder = "instruments_data"
os.makedirs(output_folder, exist_ok=True)

# Save each DataFrame to Parquet (data already standardized in previous cell)
files_created = []

# NSE + NFO instruments
df_all.to_parquet(f"{output_folder}/df_all.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/df_all.parquet")

df_equity.to_parquet(f"{output_folder}/equity.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/equity.parquet")

df_index.to_parquet(f"{output_folder}/indices.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/indices.parquet")

df_futures.to_parquet(f"{output_folder}/futures.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/futures.parquet")

df_options_ce.to_parquet(f"{output_folder}/options_ce.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/options_ce.parquet")

df_options_pe.to_parquet(f"{output_folder}/options_pe.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/options_pe.parquet")

# MCX dataframes (already created and standardized in previous cell)
df_mcx.to_parquet(f"{output_folder}/mcx_all.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_all.parquet")

df_mcx_futures.to_parquet(f"{output_folder}/mcx_futures.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_futures.parquet")

df_mcx_ce.to_parquet(f"{output_folder}/mcx_options_ce.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_options_ce.parquet")

df_mcx_pe.to_parquet(f"{output_folder}/mcx_options_pe.parquet", engine='pyarrow')
files_created.append(f"{output_folder}/mcx_options_pe.parquet")

print("✅ Parquet files created successfully!")
print(f"\n📊 Summary:")
print(f"   - NSE/NFO Total: {len(df_all):,} instruments")
print(f"   - MCX Total: {len(df_mcx):,} instruments")
print(f"   - MCX Futures: {len(df_mcx_futures):,} instruments")
print(f"   - MCX CE: {len(df_mcx_ce):,} instruments")
print(f"   - MCX PE: {len(df_mcx_pe):,} instruments")

print("\n📁 Files saved to:")
for file in files_created:
    full_path = os.path.abspath(file)
    file_size = os.path.getsize(full_path) / (1024 * 1024)  # Size in MB
    print(f"   - {file} ({file_size:.2f} MB)")
    
print("\n💡 To view in Python:")
print("   import pandas as pd")
print("   df = pd.read_parquet('instruments_data/equity.parquet')")

✅ Parquet files created successfully!

📊 Summary:
   - NSE/NFO Total: 48,685 instruments
   - MCX Total: 73,774 instruments
   - MCX Futures: 145 instruments
   - MCX CE: 36,809 instruments
   - MCX PE: 36,809 instruments

📁 Files saved to:
   - instruments_data/df_all.parquet (1.13 MB)
   - instruments_data/equity.parquet (0.35 MB)
   - instruments_data/indices.parquet (0.01 MB)
   - instruments_data/futures.parquet (0.03 MB)
   - instruments_data/options_ce.parquet (0.54 MB)
   - instruments_data/options_pe.parquet (0.54 MB)
   - instruments_data/mcx_all.parquet (1.42 MB)
   - instruments_data/mcx_futures.parquet (0.01 MB)
   - instruments_data/mcx_options_ce.parquet (0.94 MB)
   - instruments_data/mcx_options_pe.parquet (0.94 MB)

💡 To view in Python:
   import pandas as pd
   df = pd.read_parquet('instruments_data/equity.parquet')


## 5. Create Instrument Tree (Nested Dictionary)

In [24]:
# Create a nested dictionary: { name: { type: dataframe } }
# This groups by name first, then by instrument_type within each name
instrument_tree = {
    name: {inst_type: data for inst_type, data in name_group.groupby('instrument_type')}
    for name, name_group in df_all.groupby('name')
}



In [ ]:
print(f"✅ Created instrument_tree with {len(instrument_tree)} unique instrument names")
print(f"Example: instrument_tree['BSE'] contains: {list(instrument_tree.get('BSE', {}).keys())}")

# Usage:
BSE = instrument_tree['BSE']['FUT']
pprint(BSE)

✅ Created instrument_tree with 7582 unique instrument names
Example: instrument_tree['ADANIENT'] contains: ['CE', 'EQ', 'FUT', 'PE']
     instrument_token exchange_token tradingsymbol name  last_price  \
9306         15174914          59277   BSE26FEBFUT  BSE         0.0   
9307         13269762          51835   BSE26MARFUT  BSE         0.0   
9308         17084162          66735   BSE26APRFUT  BSE         0.0   

         expiry  strike  tick_size  lot_size instrument_type  segment exchange  
9306 2026-02-24     0.0        0.1       375             FUT  NFO-FUT      NFO  
9307 2026-03-30     0.0        0.1       375             FUT  NFO-FUT      NFO  
9308 2026-04-28     0.0        0.1       375             FUT  NFO-FUT      NFO  


## 5. Option Chain Analysis

In [ ]:
print(f"✅ Created instrument_tree with {len(instrument_tree)} unique instrument names")
print(f"Example: instrument_tree['ADANIENT'] contains: {list(instrument_tree.get('BSE', {}).keys())}")      

In [ ]:
def build_option_chain(instruments, symbol, expiry):
    # Getting all options associated with symbol and expiry
    """
    instruments : list from kite.instruments("NFO")
    symbol      : e.g. 'NIFTY'
    expiry      : datetime.date
    """

    df = pd.DataFrame(instruments)

    chain = df[
        (df['name'] == symbol) &
        (df['expiry'] == expiry) &
        (df['instrument_type'].isin(['CE', 'PE']))
    ]

    return chain[['tradingsymbol', 'strike', 'instrument_type', 'expiry']]

def enrich_with_market_data(kite, option_chain_df):
    # Use exchange:tradingsymbol format for kite.quote()
    symbols = ["NFO:" + s for s in option_chain_df['tradingsymbol']]
    quotes = kite.quote(symbols)

    data = []
    for _, row in option_chain_df.iterrows():
        symbol_key = "NFO:" + row['tradingsymbol']
        q = quotes.get(symbol_key, {})
        
        # Extract bid/ask safely
        depth = q.get('depth', {})
        buy_orders = depth.get('buy', [])
        sell_orders = depth.get('sell', [])
        
        data.append({
            **row,
            'ltp': q.get('last_price'),
            'oi': q.get('oi'),
            'volume': q.get('volume'),
            'bid': buy_orders[0].get('price') if buy_orders else None,
            'ask': sell_orders[0].get('price') if sell_orders else None,
        })

    return pd.DataFrame(data)




In [11]:
# Test kite.quote() with a list of symbols
symbols = ["NIFTY26FEB24000CE", "NIFTY26FEB24000PE", "NIFTY26FEB23900CE"]

quotes = kite.quote(symbols)
pprint(quotes)

{}


In [12]:
# Load instruments
#instruments = kite.instruments("NFO")

# Parameters
symbol = "NIFTY"
expiry_date = date(2026, 2, 3)   # example expiry

# Build and enrich chain
chain = build_option_chain(instruments_nfo, symbol, expiry_date)
chain = enrich_with_market_data(kite, chain)

# Analytics
# pcr = calculate_pcr(chain)
# support, resistance = oi_support_resistance(chain)
# mp = max_pain(chain)

# print("PCR:", pcr)
# print("Support:", support)
# print("Resistance:", resistance)
# print("Max Pain:", mp)

In [13]:
chain

,tradingsymbol,strike,instrument_type,expiry,ltp,oi,volume,bid,ask
0,NIFTY2620325400CE,25400.0,CE,2026-02-03,161.80,4851795,187444790,161.40,162.30
1,NIFTY2620325400PE,25400.0,PE,2026-02-03,210.95,2530710,70448820,211.05,211.80
2,NIFTY2620325450CE,25450.0,CE,2026-02-03,140.60,1684800,80355405,139.80,140.55
3,NIFTY2620325450PE,25450.0,PE,2026-02-03,239.90,660920,16264365,239.50,240.00
4,NIFTY2620325350CE,25350.0,CE,2026-02-03,186.35,3057080,155775295,186.30,186.85
...,...,...,...,...,...,...,...,...,...
189,NIFTY2620323400PE,23400.0,PE,2026-02-03,4.55,3141385,13960570,4.60,4.80
190,NIFTY2620323350CE,23350.0,CE,2026-02-03,2073.20,130,0,1836.65,2124.40
191,NIFTY2620323350PE,23350.0,PE,2026-02-03,4.30,655005,6699095,4.30,4.50
192,NIFTY2620323300CE,23300.0,CE,2026-02-03,1802.00,130,0,1992.30,2196.25


## 6. Fetch Current Positions

Positions Analyser

In [54]:
# 1. Fetch and Combine
positions = kite.positions()
pprint(positions)

{'day': [],
 'net': [{'average_price': 2628.166667,
          'buy_m2m': 21615,
          'buy_price': 2628.166667,
          'buy_quantity': 3,
          'buy_value': 78845.00001,
          'close_price': 720.5,
          'day_buy_price': 0,
          'day_buy_quantity': 0,
          'day_buy_value': 0,
          'day_sell_price': 0,
          'day_sell_quantity': 0,
          'day_sell_value': 0,
          'exchange': 'MCX',
          'instrument_token': 131565063,
          'last_price': 720.5,
          'm2m': 0,
          'multiplier': 10,
          'overnight_price': 0,
          'overnight_quantity': 3,
          'overnight_value': 78845.00001,
          'pnl': -57230.00001,
          'product': 'NRML',
          'quantity': 3,
          'realised': 0,
          'segment': '',
          'sell_m2m': 0,
          'sell_price': 0,
          'sell_quantity': 0,
          'sell_value': 0,
          'settlement': False,
          'tradingsymbol': 'GOLDM26FEB148000PE',
          'unrea

## 7. Trailing Stop Loss Implementation

In [ ]:
# Function to implement trailing stop loss
def trailing_stop_loss(position_symbol, stop_loss_percent, kite_instance, db_connection):
    """
    Monitors a position and triggers a stop loss order when price drops by the specified percentage.
    
    Args:
        position_symbol: Trading symbol (e.g., 'NSE:RELIANCE')
        stop_loss_percent: Stop loss percentage (e.g., 10 for 10%)
        kite_instance: Kite object for API calls
        db_connection: SQLite database connection
    
    Returns:
        Dictionary with order details if triggered, None otherwise
    """
    
    try:
        # 1. Get current LTP
        ltp_data = kite_instance.ltp(position_symbol)
        current_ltp = ltp_data[position_symbol]['last_price']
        
        # 2. Find position in pos_combined
        position = pos_with_finite_max[pos_with_finite_max['tradingsymbol'] == position_symbol]
        
        if position.empty:
            print(f"❌ Position not found for {position_symbol}")
            return None
        
        avg_price = position['average_price'].values[0]
        current_qty = position['quantity'].values[0]
        
        # 3. Calculate stop loss trigger price
        stop_loss_price = avg_price * (1 - stop_loss_percent / 100)
        
        # 4. Check if current price has breached stop loss
        print(f"\n📊 {position_symbol}")
        print(f"   Average Price: ₹{avg_price:.2f}")
        print(f"   Current LTP: ₹{current_ltp:.2f}")
        print(f"   Stop Loss Trigger: ₹{stop_loss_price:.2f} ({stop_loss_percent}%)")
        print(f"   Current Qty: {current_qty}")
        
        if current_ltp <= stop_loss_price:
            print(f"\n🚨 STOP LOSS TRIGGERED for {position_symbol}!")
            
            # 5. Save to database before triggering order
            cursor = db_connection.cursor()
            cursor.execute('''
                INSERT INTO stop_loss_events (symbol, avg_price, trigger_price, current_ltp, quantity, percentage, status, timestamp)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (position_symbol, avg_price, stop_loss_price, current_ltp, current_qty, stop_loss_percent, 'TRIGGERED', 
                  datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
            db_connection.commit()
            
            # 6. Place exit order (sell for long position)
            try:
                order = kite_instance.place_order(
                    variety="regular",
                    exchange=position['exchange'].values[0],
                    tradingsymbol=position_symbol,
                    transaction_type="SELL" if current_qty > 0 else "BUY",
                    quantity=abs(current_qty),
                    order_type="MARKET",
                    product="MIS"
                )
                
                print(f"✅ Stop Loss Order Placed! Order ID: {order}")
                
                # Log to database
                cursor.execute('''
                    INSERT INTO stop_loss_orders (symbol, order_id, order_type, quantity, price, status)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (position_symbol, order, "MARKET", abs(current_qty), current_ltp, 'PLACED'))
                db_connection.commit()
                
                return {
                    'symbol': position_symbol,
                    'order_id': order,
                    'trigger_price': stop_loss_price,
                    'exit_price': current_ltp,
                    'quantity': abs(current_qty),
                    'loss_percent': stop_loss_percent
                }
            
            except Exception as e:
                print(f"❌ Failed to place order: {str(e)}")
                return None
        
        else:
            print(f"✅ Price is above stop loss level")
            return None
    
    except Exception as e:
        print(f"❌ Error in trailing_stop_loss: {str(e)}")
        return None


# Create stop_loss tables in database (run once)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS stop_loss_events (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        symbol TEXT NOT NULL,
        avg_price REAL,
        trigger_price REAL,
        current_ltp REAL,
        quantity INTEGER,
        percentage REAL,
        status TEXT,
        timestamp DATETIME
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS stop_loss_orders (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        symbol TEXT NOT NULL,
        order_id TEXT,
        order_type TEXT,
        quantity INTEGER,
        price REAL,
        status TEXT,
        created_at DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')

conn.commit()
print("✅ Stop Loss tables created successfully!")

# Example usage:
# trailing_stop_loss('NSE:RELIANCE', 10, kite, conn)
# trailing_stop_loss('NIFTY26FEBFUT', 5, kite, conn)

## 8. Position Summary Analysis

In [56]:
pos_day = pd.DataFrame(positions['day'])
pos_net = pd.DataFrame(positions['net'])
pos_combined = pd.concat([pos_day, pos_net], ignore_index=True)

# 2. Fix Missing Columns (Safety Check)
# If 'expiry' or 'strike' don't exist, create them as empty strings
for col in ['expiry', 'strike', 'instrument_type']:
    if col not in pos_combined.columns:
        pos_combined[col] = ""

# 3. Clean up Expiry (Only if it contains data)
pos_combined['expiry'] = pd.to_datetime(pos_combined['expiry'], errors='coerce')

# 4. Create the "Goddamn" Summary
# We include expiry, strike, and type for a complete breakdown
pos_summary = pos_combined.groupby(['tradingsymbol', 'expiry', 'strike', 'instrument_type']).agg({
    'quantity': 'sum',
    'pnl': 'sum',
    'average_price': 'mean',
    'last_price': 'first',
    'm2m': 'sum'
}).reset_index()

# 5. Add Key Stats
# Exposure value (Market Value)
pos_summary['exposure'] = pos_summary['quantity'] * pos_summary['last_price']

# P&L Percentage (Safety check for 0 average price)
pos_summary['pnl_pct'] = 0.0
mask = (pos_summary['average_price'] != 0) & (pos_summary['quantity'] != 0)
pos_summary.loc[mask, 'pnl_pct'] = (pos_summary['pnl'] / (pos_summary['average_price'] * pos_summary['quantity'].abs())) * 100

# 6. Save to Parquet
pos_summary.to_parquet('pos_xyz_summary.parquet', engine='pyarrow')

print("Summary Generated Successfully!")
print(pos_summary[['tradingsymbol', 'pnl', 'pnl_pct', 'exposure']].head())

Summary Generated Successfully!
Empty DataFrame
Columns: [tradingsymbol, pnl, pnl_pct, exposure]
Index: []


## 9. Group Positions by Base Symbol and Expiry

In [57]:
# Group positions by base symbol (name) and expiry, then merge with master df_all

# Step 1: Merge positions with df_all to get the 'name' (base symbol)
pos_enriched = pos_combined.merge(
    df_all[['tradingsymbol', 'name', 'expiry', 'instrument_type', 'strike', 'lot_size']],
    on='tradingsymbol',
    how='left',
    suffixes=('', '_master')
)

# Use the master expiry if position expiry is missing
if 'expiry_master' in pos_enriched.columns:
    pos_enriched['expiry'] = pos_enriched['expiry'].fillna(pos_enriched['expiry_master'])

# Step 2: Group by base name and expiry
position_groups = pos_enriched.groupby(['name', 'expiry']).agg({
    'tradingsymbol': 'count',  # Count of positions
    'quantity': 'sum',
    'pnl': 'sum',
    'm2m': 'sum',
    'average_price': 'mean',
    'last_price': 'mean',
    'instrument_type': lambda x: x.unique().tolist()  # List unique instrument types
}).reset_index()

# Rename for clarity
position_groups.rename(columns={'tradingsymbol': 'position_count'}, inplace=True)

# Calculate total exposure
position_groups['total_exposure'] = position_groups['quantity'] * position_groups['last_price']

print("="*80)
print("POSITIONS GROUPED BY BASE SYMBOL AND EXPIRY")
print("="*80)
print(position_groups)
print(f"\nTotal unique base symbols with positions: {position_groups['name'].nunique()}")
print(f"Total position groups: {len(position_groups)}")

POSITIONS GROUPED BY BASE SYMBOL AND EXPIRY
          name     expiry  position_count  quantity           pnl  m2m  \
0     ADANIENT 2026-02-24               1      -309   6952.500000    0   
1   ADANIGREEN 2026-02-24               2     -1200   1350.000000    0   
2     HDFCBANK 2026-02-24               1     -1100  -1842.500000    0   
3          IEX 2026-02-24               2     -7500    -37.500000    0   
4         INFY 2026-02-24               2      -800    420.000000    0   
5          ITC 2026-02-24               2     -8000  -7760.001600    0   
6   KALYANKJIL 2026-02-24               2     -3525 -16920.000000    0   
7   MIDCPNIFTY 2026-02-24               1      -120   -132.000000    0   
8        NIFTY 2026-02-10               1        65    143.000000    0   
9        NIFTY 2026-02-24               1        65   -949.000000    0   
10       NIFTY 2026-03-30               3      -195 -20751.250065    0   
11   TATAELXSI 2026-02-24               2      -200  22400.000000   

## 10. Detailed Position View by Base Symbol

In [58]:
# Detailed view: Show all positions for each base symbol grouped by expiry

print("\n" + "="*80)
print("DETAILED POSITIONS BY BASE SYMBOL")
print("="*80)

for name in pos_enriched['name'].unique():
    name_positions = pos_enriched[pos_enriched['name'] == name].copy()
    
    # Sort by expiry and instrument type
    name_positions = name_positions.sort_values(['expiry', 'instrument_type_master'])
    
    print(f"\n{'='*80}")
    print(f"📊 {name}")
    print(f"{'='*80}")
    
    # Show key columns
    display_cols = ['tradingsymbol', 'instrument_type_master', 'expiry', 'strike_master', 
                    'quantity', 'average_price', 'last_price', 'pnl', 'm2m']
    
    # Filter to existing columns
    display_cols = [col for col in display_cols if col in name_positions.columns]
    
    print(name_positions[display_cols].to_string(index=False))
    
    # Summary for this symbol
    total_qty = name_positions['quantity'].sum()
    total_pnl = name_positions['pnl'].sum()
    total_m2m = name_positions['m2m'].sum()
    
    print(f"\n  📈 Total Quantity: {total_qty}")
    print(f"  💰 Total P&L: ₹{total_pnl:,.2f}")
    print(f"  📊 Total M2M: ₹{total_m2m:,.2f}")


DETAILED POSITIONS BY BASE SYMBOL

📊 nan
Empty DataFrame
Columns: [tradingsymbol, instrument_type_master, expiry, strike_master, quantity, average_price, last_price, pnl, m2m]
Index: []

  📈 Total Quantity: 0
  💰 Total P&L: ₹0.00
  📊 Total M2M: ₹0.00

📊 ADANIENT
      tradingsymbol instrument_type_master     expiry  strike_master  quantity  average_price  last_price    pnl  m2m
ADANIENT26FEB1800PE                     PE 2026-02-24         1800.0      -309          52.65       30.15 6952.5    0

  📈 Total Quantity: -309
  💰 Total P&L: ₹6,952.50
  📊 Total M2M: ₹0.00

📊 ADANIGREEN
       tradingsymbol instrument_type_master     expiry  strike_master  quantity  average_price  last_price    pnl  m2m
ADANIGREEN26FEB900CE                     CE 2026-02-24          900.0      -600          17.40       15.05 1410.0    0
ADANIGREEN26FEB620PE                     PE 2026-02-24          620.0      -600           3.55        3.65  -60.0    0

  📈 Total Quantity: -1200
  💰 Total P&L: ₹1,350.00
  📊 T

## Red/Green/Yellow Position Classification

## 11. Classify Positions (Red/Green/Yellow)

In [ ]:
# Classify positions as Red (losing), Green (profit), or Yellow (breakeven/small moves)
# Red: P&L < -1000
# Green: P&L > 1000
# Yellow: -1000 <= P&L <= 1000

# Create masks for classification
mask_red = pos_combined['pnl'] < -1000
mask_green = pos_combined['pnl'] > 1000
mask_yellow = (pos_combined['pnl'] >= -1000) & (pos_combined['pnl'] <= 1000)

# Create the dataframes
df_red = pos_combined[mask_red].copy()
df_green = pos_combined[mask_green].copy()
df_yellow = pos_combined[mask_yellow].copy()

print("="*80)
print("POSITION CLASSIFICATION BY P&L")
print("="*80)
print(f"\n🔴 RED (Loss > ₹1000): {len(df_red)} positions")
print(f"   Total P&L: ₹{df_red['pnl'].sum():,.2f}")
if len(df_red) > 0:
    print(f"   Avg P&L: ₹{df_red['pnl'].mean():,.2f}")
    print(f"   Worst: ₹{df_red['pnl'].min():,.2f}")

print(f"\n🟡 YELLOW (Between -₹1000 and +₹1000): {len(df_yellow)} positions")
print(f"   Total P&L: ₹{df_yellow['pnl'].sum():,.2f}")
if len(df_yellow) > 0:
    print(f"   Avg P&L: ₹{df_yellow['pnl'].mean():,.2f}")

print(f"\n🟢 GREEN (Profit > ₹1000): {len(df_green)} positions")
print(f"   Total P&L: ₹{df_green['pnl'].sum():,.2f}")
if len(df_green) > 0:
    print(f"   Avg P&L: ₹{df_green['pnl'].mean():,.2f}")
    print(f"   Best: ₹{df_green['pnl'].max():,.2f}")

print(f"\n{'='*80}")
print(f"OVERALL SUMMARY")
print(f"{'='*80}")
print(f"Total Positions: {len(pos_combined)}")
print(f"Net P&L: ₹{pos_combined['pnl'].sum():,.2f}")

## 12. Display Red Positions (Losses)

In [ ]:
# Display red positions in detail
if len(df_red) > 0:
    print("\n" + "="*80)
    print("🔴 RED POSITIONS (Losses > ₹1000)")
    print("="*80)
    display_cols = ['tradingsymbol', 'quantity', 'average_price', 'last_price', 'pnl', 'm2m']
    display_cols = [col for col in display_cols if col in df_red.columns]
    print(df_red[display_cols].sort_values('pnl').to_string(index=False))

## 13. Display Green Positions (Profits)

In [ ]:
# Display green positions in detail
if len(df_green) > 0:
    print("\n" + "="*80)
    print("🟢 GREEN POSITIONS (Profits > ₹1000)")
    print("="*80)
    display_cols = ['tradingsymbol', 'quantity', 'average_price', 'last_price', 'pnl', 'm2m']
    display_cols = [col for col in display_cols if col in df_green.columns]
    print(df_green[display_cols].sort_values('pnl', ascending=False).to_string(index=False))

## 14. SQLite Database Setup

In [ ]:
import sqlite3
import os

# Create a SQLite database (creates file if it doesn't exist)
db_path = "trading_data.db"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"Connected to SQLite database: {os.path.abspath(db_path)}")

Connected to SQLite database: c:\Users\sathy\OneDrive\Desktop\Project Algoarms\pykite\trading_data.db


## 15. Create Database Tables

In [ ]:
# Create a sample table for storing trades
cursor.execute('''
    CREATE TABLE IF NOT EXISTS trades (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        symbol TEXT NOT NULL,
        trade_type TEXT NOT NULL,
        quantity INTEGER NOT NULL,
        price REAL NOT NULL,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')

# Create a table for storing holdings snapshots
cursor.execute('''
    CREATE TABLE IF NOT EXISTS holdings_snapshot (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        symbol TEXT NOT NULL,
        quantity INTEGER,
        average_price REAL,
        last_price REAL,
        pnl REAL,
        snapshot_time DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')

conn.commit()
print("Tables created successfully!")

Tables created successfully!


## 16. Insert Sample Trade Data

In [ ]:
# Example: Insert a sample trade
cursor.execute('''
    INSERT INTO trades (symbol, trade_type, quantity, price)
    VALUES (?, ?, ?, ?)
''', ('NSE:RELIANCE', 'BUY', 10, 2450.50))

conn.commit()

# Query all trades
cursor.execute('SELECT * FROM trades')
trades = cursor.fetchall()
print("All trades:")
for trade in trades:
    print(trade)

All trades:
(1, 'NSE:RELIANCE', 'BUY', 10, 2450.5, '2026-01-29 00:46:12')
(2, 'NSE:RELIANCE', 'BUY', 10, 2450.5, '2026-01-29 02:02:52')


## 17. Database Helper Functions

In [ ]:
# Helper function to close the database connection
def close_db():
    conn.close()
    print("Database connection closed.")

# Uncomment to close when done:
# close_db()

## 18. Get LTP and Quotes

In [ ]:
# Get LTP (Last Traded Price) for a single stock
ltp = kite.ltp("NSE:RELIANCE")
print(ltp)

# Or get LTP for multiple instruments
ltps = kite.ltp(["NSE:RELIANCE", "NSE:INFY", "NSE:TCS"])
print(ltps)

# Get detailed quote with more data
quote = kite.quote("NSE:RELIANCE")
print(quote)

# Get multiple quotes
quotes = kite.quote(["NSE:RELIANCE", "NSE:INFY"])
print(quotes)


PermissionException: Insufficient permission for that call.

## 19. API Rate Limit Test

In [61]:
import time
from datetime import datetime
from IPython.display import clear_output

# API Rate Limit Test
counter = 1
try:
    while True:
        start_time = time.time()
        
        # Make API calls
        positions = kite.positions()
        orders = kite.orders()
        
        end_time = time.time()
        elapsed = end_time - start_time
        
        # Clear previous output and print current stats
        clear_output(wait=True)
        print(f"Request #{counter}")
        print(f"Time: {datetime.now().strftime('%H:%M:%S')}")
        print(f"Response Time: {elapsed:.3f} seconds")
        print(f"Positions retrieved: {len(positions.get('net', []))} net, {len(positions.get('day', []))} day")
        print(f"Orders retrieved: {len(orders)}")
        print("\nPress Ctrl+C to stop...")
        
        counter += 1
        
        # Sleep for 0.1 second before next request
        time.sleep(0.1)
        
except KeyboardInterrupt:
    print(f"\n\nStopped after {counter-1} requests")
except Exception as e:
    print(f"\n\nError occurred after {counter-1} requests:")
    print(f"Error Type: {type(e).__name__}")
    print(f"Error Message: {str(e)}")
    print(f"Time: {datetime.now().strftime('%H:%M:%S')}")

Request #45
Time: 03:34:52
Response Time: 0.446 seconds
Positions retrieved: 35 net, 0 day
Orders retrieved: 0

Press Ctrl+C to stop...


Stopped after 45 requests
